# Week 3 - NLP and Deep Learning

---

# Lecture 5 - Language Identification with a Feedforward Neural Network


In this exercise, you will implement the forward step of a FFNN from scratch and compare your solution to Pytorch on a small toy example to predict the language for a given word. 

It is very important that you understand the basic building blocks (input/output: how to encode your instances, the labels; the model: what the neural network consists of, how to learn its weights, how to do a forward pass for prediction). 

##  1. Representing the data

We are assuming multi-class classification tasks for the assignments of this week. The labels are: $$ y \in \{da,nl,en\}$$

We will use the same data as in week2, from:
* English [Wookipedia](https://starwars.fandom.com/wiki/Main_Page)  
* Danish [Kraftens Arkiver](https://starwars.fandom.com/da/wiki) 
* Dutch [Yodapedia](https://starwars.fandom.com/da/wiki)


In [ ]:
def load_langid(path):
    text = []
    labels = []
    for line in open(path, encoding="utf-8"):
        tok = line.strip().split('\t')
        labels.append(tok[0])
        text.append(tok[1])
    return text, labels

wooki_train_text, wooki_train_labels = load_langid('langid-data/wookipedia_langid.train.tok.txt')


* a): Convert the training data into n-hot format, where each feature represents whether a **single character** is present or not.  Similarly, convert the labels into numeric format. For simplicity, you can assume a closed vocabulary (only the letters in wookie_train_text, no unknown-character handling). Keep original casing, and assign the character indices based on their chronological order.

  * What is the vocabulary size?
  
**Hint:** It is easier for the rest of the assignment if you directly use a torch tensor to save the features ([tutorial](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py)), a 2d torch tensor filled with 0's can be initiated with: `torch.zeros(dim1, dim2, dtype=float)`. Note the use of `float` instead of `int` here, which is only because the `torch.mm` requires float tensors as input.

##  2: Forward pass (from scratch)

### Feedforward Neural Networks (FNNs) or MLPs

Feedforward Neural Networks (FNNs) are also called Multilayer Perceptrons (MLPs). These are the most basic types of neural networks. They are called this way as the information is flowing from the input nodes through the network up to the output nodes. 

It is essential to understand that a neural network is a non-linear classification model which is based upon function application. Each layer in a neural network is an application of a function.

Summary (by J.Frellsen):
<img src="pics/fnn_jf.png">

You are going to implement the forward step manually on a small dataset. You will create a network following the design in the following figure (note that the input should be the sames size as the number of characters found in the previous assignment, instead of 4):

<img src="pics/nn.svg">

a) How many neurons do hidden layer 1 and hidden layer 2 have? Note: the bias node is not shown in the figure, you do not have to count them for this assignment.

b) How many neurons does the output layer have? And the input layer? (Note: the figure shows only 4 input nodes, in this example your input size is defined in the previous assignment - what is the input layer size?)

c) Specify the size of layers of the feedforward neural network:

In [ ]:
## helper functions to determine the input and output dimensions of each layer
input_dim = 

hidden_dim1 = 
hidden_dim2 = 

output_dim = 

d) Now initialize the layers themselves as torch tensors (do not use a torch.module here!). You can define the bias and the weights in separate tensors. The weights should be initialized randomly (`torch.randn((dim1, dim2), dtype=torch.float)`, see also [torch.randn](https://pytorch.org/docs/stable/generated/torch.randn.html)) and the biases can be set to 1 (`torch.ones(dim1, dtype=torch.float)`, see also [torch.ones](https://pytorch.org/docs/stable/generated/torch.ones.html)). Confirm whether their size match the answer to `b)` and `a)` by printing .shape of the tensors.


In [ ]:
## define all parameters of this NN


Now that we have defined the shape of all parameters, we are ready to "connect the dots" and build the network. 

It is instructive to break the computation of each layer down into two steps: the scores $a1$ are obtained by the linear function followed by the activation applications $\sigma$ to obtain the representation $z1$, as in:

$$ a1 = xW_1 + b_1$$
$$ z1 = \sigma(a1)$$

d) Specify the entire network up to the output layer $z3$, and **up to and exclusive** the final application of the softmax, the last activation function, which is provided. For multiplication [torch.mm](https://pytorch.org/docs/stable/generated/torch.mm.html) can be used. Use a tanh activation function: [torch.tanh](https://pytorch.org/docs/stable/generated/torch.tanh.html).

The exact implementation of the softmax might differ from toolkit to toolkit (due to variations in implementation details in order to obtain numerical stability). Therefore, we will use the Pytorch implementation for the softmax calculation ([torch.nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)).

In [ ]:
## implement the forward pass (up to and exclusive the softmax) 
## apply it to the training data `data_train` - use vectorization


We can check that all predictions sum up to approximately 1 (hint: use `torch.sum` with `axis=1`)




Congrats! you have made it through the manual construction of the forward pass. Note that these weights are still random, so performance is not expected to be good. Now lets compare your implementation to a set of pre-determined weights.

##  3. Where do the weights come from?  Loading existing weights

So far, the model that you used randomly initialized weights. In this step we will load pre-trained model weights and do the forward pass with those weights, in order to check your implementation against model predictions computed by the toolkit.

Now we are going to:
* load pretrained weights for all parameters
* apply the weights to the evaluation data
* check that your manual softmax scores match the ones obtained by the pre-trained model `model` that we will load
* convert the output to labels and calculate the accuracy score

First, lets load the pre-trained model:

In [ ]:
import torch
import torch.nn as nn

# use the character indexing from assignment 1
idx2char = ['H', 'e', ' ', 'v', 'n', 'w', 't', 's', 'o', 'f', 'a', 'r', 'u', 'g', 'h', ',', 'i', 'c', 'y', 'd', 'b', 'm', 'p', 'l', 'k', '.', 'D', 'E', 'C', 'j', 'R', 'S', 'U', '1', "'", 'æ', 'å', 'q', '`', 'I', '(', ')', 'M', 'F', '-', 'x', 'K', '9', '5', 'B', 'W', 'z', 'G', 'P', 'L', '/', 'O', '6', 'T', '7', 'Z', '2', '0', 'J', 'V', 'A', 'ø', 'X', '–', 'N', 'ë', ':', '&', '3', 'Y', 'é', '4', '[', ']', '’', ';', '8', 'É', 'Æ', 'Q', '!', '—', 'ï', '°', 'ō', '\u200b', '‘', 'ń', '“', '”', '?', 'Å', '<', '>', '#', '%', '+', 'ʊ', 'ɹ', 'ə', 'ɑ', 'ö', 'à', 'á', 'è', '=', 'ü', 'Ø', '∑', '^', 'ś', 'ñ', '|', '½', '$', '«', '™', 'ó', '´', '…', '―', '»', 'ː', 'θ', '²', 'Θ']
char2idx = {'H': 0, 'e': 1, ' ': 2, 'v': 3, 'n': 4, 'w': 5, 't': 6, 's': 7, 'o': 8, 'f': 9, 'a': 10, 'r': 11, 'u': 12, 'g': 13, 'h': 14, ',': 15, 'i': 16, 'c': 17, 'y': 18, 'd': 19, 'b': 20, 'm': 21, 'p': 22, 'l': 23, 'k': 24, '.': 25, 'D': 26, 'E': 27, 'C': 28, 'j': 29, 'R': 30, 'S': 31, 'U': 32, '1': 33, "'": 34, 'æ': 35, 'å': 36, 'q': 37, '`': 38, 'I': 39, '(': 40, ')': 41, 'M': 42, 'F': 43, '-': 44, 'x': 45, 'K': 46, '9': 47, '5': 48, 'B': 49, 'W': 50, 'z': 51, 'G': 52, 'P': 53, 'L': 54, '/': 55, 'O': 56, '6': 57, 'T': 58, '7': 59, 'Z': 60, '2': 61, '0': 62, 'J': 63, 'V': 64, 'A': 65, 'ø': 66, 'X': 67, '–': 68, 'N': 69, 'ë': 70, ':': 71, '&': 72, '3': 73, 'Y': 74, 'é': 75, '4': 76, '[': 77, ']': 78, '’': 79, ';': 80, '8': 81, 'É': 82, 'Æ': 83, 'Q': 84, '!': 85, '—': 86, 'ï': 87, '°': 88, 'ō': 89, '\u200b': 90, '‘': 91, 'ń': 92, '“': 93, '”': 94, '?': 95, 'Å': 96, '<': 97, '>': 98, '#': 99, '%': 100, '+': 101, 'ʊ': 102, 'ɹ': 103, 'ə': 104, 'ɑ': 105, 'ö': 106, 'à': 107, 'á': 108, 'è': 109, '=': 110, 'ü': 111, 'Ø': 112, '∑': 113, '^': 114, 'ś': 115, 'ñ': 116, '|': 117, '½': 118, '$': 119, '«': 120, '™': 121, 'ó': 122, '´': 123, '…': 124, '―': 125, '»': 126, 'ː': 127, 'θ': 128, '²': 129, 'Θ': 130}

# the label indexes that were used during training
label2idx = {'da':0, 'nl':1, 'en':2}
idx2label = ['da', 'nl', 'en']

# This is the definition of an FNN model in PyTorch, and can mostly be ignored for now.
# We will focus on how to create Torch models in lecture 6
class LangId(nn.Module):
    def __init__(self, vocab_size):
        super(LangId, self).__init__()
        self.input = nn.Linear(vocab_size, 15)
        self.hidden1 = nn.Linear(15, 20)
        self.hidden2 = nn.Linear(20, 3)

    def forward(self, x):
        x = torch.tanh(self.input(x))
        x = torch.tanh(self.hidden1(x))
        x = self.hidden2(x)
        return x

lang_classifier = torch.load('model.pt')


Inspect the weights you just loaded using the `state_dict()` function of the model: 

In [ ]:
print(lang_classifier)
lang_classifier.state_dict()


* a) Convert the following dev data into the input format for the neural network above. 

**Hint** The indices of the characters are based on the order in the training data, and should match in the development data, we provide the correct idx2char and char2idx that were used to train the model in the code above.

In [ ]:
wooki_dev_text, wooki_dev_labels = load_langid('langid-data/wookipedia_langid.dev.tok.txt')

* b) run a forward pass on the dev-data with `lang_classifier`, using the forward() function

* c) Apply your manual implementation of the forward pass to the evaluation data by using the parameters (weights) you just loaded with `state_dict()`. This allows you to check if you get the same results back as the model implemented in Torch. If the outputs match, you implemented the forward pass correctly, congratulations!

**Hint**: internally the torch model saves the weight in a transposed vector for efficiency reasons. This means that W1 will have the dimension of (15,131). To use your previous implementation you have to call the the transpose function in Pytorch ([`.t()`](https://pytorch.org/docs/stable/generated/torch.t.html)), which will convert the shape to be (131,15)

* d) Now apply softmax on the resulting weights and convert the output to the label predictions.

In [ ]:
# starting point for c)
W1 = lang_classifier.state_dict()['input.weight'].t()


# Lecture 6: Word2vec and PyTorch
In the following exercises, you are going to explore what is represented with word embeddings. You are going to make use of the python gensim package and two sets of pre-trained embeddings. The embeddings can be downloaded from:

* http://itu.dk/people/robv/data/embeds/twitter.bin.tar.gz
* http://itu.dk/people/robv/data/embeds/GoogleNews-50k.bin.tar.gz

The first embeddings are skip-gram embeddings trained on a collection of 2 billion words from English tweets collected during 2012 and 2018 with the default settings of word2vec. The second embeddings are trained on 100 billion words from Google News. They have both been truncated to the most frequent 500,000 words. Note that loading that each of these embeddings require approximately 2GB of ram.

The embeddings can be loaded in gensim as follows:

In [ ]:
import gensim.models

twitEmbs = gensim.models.KeyedVectors.load_word2vec_format(
                                'twitter.bin', binary=True)
print('loading finished')

You can now use the index operator ``[]`` or the function ``get_vector()`` to acces the individual word embeddings.

In [ ]:
twitEmbs['cat']

## 4. Word similarities
Cosine distance can be used to measure the distance between two words. It is defined as:
\begin{equation}
cos_{\vec{a},\vec{b}} = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}| |\vec{b}|} = \frac{\sum^n_1 a_i b_i}{\sqrt{\sum^n_1 a_i^2} \sqrt{\sum^n_1 b_i^2}}
\end{equation}

* a) Implement the cosine similarity using pure python (using only the ``math`` package, no other libraries). Note that `similarity == 1-distance`.

You can compare your scores to the gensim implementation to check wheter it is correct. The following code should give the same output

In [ ]:
def cosine(vec1, vec2):
    pass

print(twitEmbs.distance('cat', 'dog'))
print(cosine(twitEmbs['cat'], twitEmbs['dog']))


In wordnet, the distance between two senses can be based on the distance in the taxonomy. The most common metric for this is:

* Wu-Palmer Similarity: denotes how similar two word senses are, based on the depth of the two senses in the taxonomy and of their Least Common Subsumer (most specific ancestor node).

It can be obtained in python like this:

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet

first_word = wordnet.synsets('cat')[0] #0 means: most common sense
second_word = wordnet.synsets('dog')[0]
print('WordNet similarity: ' + str(first_word.wup_similarity(second_word)))

print('Twitter similarity: ' + str(twitEmbs.similarity('cat', 'dog')))




* b) Think of 5 word pairs which have a high similarity according to you. Estimate the difference between these pairs in wordnet as well as in the Twitter embeddings and the Google News embeddings. Which method is closest to your own intuition? (You can use the gensim implementation of cosine similarity here.)


## 5. Training a Language Identification model using PyTorch

#### **Objective**  
Implement a PyTorch-based model for **language identification**. The goal is to train a model that can classify input text into three different languages.

* a) **Prepare Training Data.** 

Prepare the training data (texts) in an n-hot format, and the labels in a seperate matrix. The training data will be of shape (15000, 131) for 15000 instances and 131 features (i.e. characters). The train labels will be of shape (15000), and contains labels representing the languages (e.g. 0,1,2).

In [ ]:
train_data = # n-hot format training data obtained in 1a
train_labels = # numeric format labels obtained in 1a

* b) **Initialize Model, Loss Function, and Optimizer**

You can use the model as defined in assignment 3.

In [ ]:
batch_size =
train_dataset =
train_loader =

* c) **Prepare dataset and dataloader**

You can use torch.utils.data.DataLoader and torch.utils.data.TensorDataset

In [ ]:
langClassifier = # Initialize langClassifier using LangId with vocab size as an argument
criterion = # CrossEntropyLoss function
optimizer = # SGD optimizer with learning rate 0.00001 and momentum 0.9

* d) **Implement the training Loop**

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times

    for batch_data, batch_labels in train_loader:  # iterate over batches

        # Zero the parameter gradients

        # Run the forward pass on langClassifier model with batch_data and collect the outputs

        # Calculate the loss of the outputs with respect to batch_labels

        # Perform backpropagation of the loss

        # Update model weights using the optimizer

    # Print the loss at the end of each epoch.

* e) **Save the Model**

Save the trained langClassifier model using torch.save()

* f) **Load and Prepare Development Set**

In [ ]:
# Load dev_text and dev_labels from 'langid-data/wookipedia_langid.dev.tok.txt'

dev_data = # n-hot format data similar to that obtained in 3a
dev_labels = # numeric format labels similar to that obtained in 3a

* g) **Evaluate the Model**
Evaluate the results on dev data:

In [ ]:
logit = # Forward pass on dev_data with langClassifier model
predicted = # Obtain the prediction with the highest logit; Hint: argmax

# Iterate through predicted and dev_labels and count the correct labels

# Compute and print accuracy

* h) Explore the tuning of one part of the model to improve performance, you can choose for example to tune the learning rate, change the number of layers, dimensions of layers, or prune the input space.